<a href="https://colab.research.google.com/github/Gonzalo-Messina/Programacion-Concurrente/blob/main/M6_threads_cpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ejemplo threads en C++

In [ ]:
%%writefile threads.cpp
#include <iostream>
#include <vector>
#include <random>
#include <thread>

std::vector<std::vector<int>> generate_random_matrix(int N, int min_value, int max_value) 
{
    //Generamos un numero realmente aleatorio con el hardware del dispositivo
    std::vector<std::vector<int>> matrix(N, std::vector<int>(N));
    std::random_device rd;
    std::mt19937 gen(rd());
    std::uniform_int_distribution<> dis(min_value, max_value);
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            matrix[i][j] = dis(gen);
        }
    }
    return matrix;
}

std::vector<std::vector<int>> matrix_multiplication(const std::vector<std::vector<int>>& matrixA, const std::vector<std::vector<int>>& matrixB) 
{
    //Algoritmo estandar para multiplicar secuencialmente dos matrices
    int N = matrixA.size();
    std::vector<std::vector<int>> result(N, std::vector<int>(N));
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            int sum = 0;
            for (int k = 0; k < N; k++) {
                sum += matrixA[i][k] * matrixB[k][j];
            }
            result[i][j] = sum;
        }
    }
    return result;
}

std::vector<std::vector<int>> concurrent_matrix_multiplication(const std::vector<std::vector<int>>& matrixA, const std::vector<std::vector<int>>& matrixB) 
{
    //Inicializamos las variables
    int N = matrixA.size();
    std::vector<std::thread> threads;
    std::vector<std::vector<int>> result(N, std::vector<int>(N));

    //Guardamos una funcion anonima en calculate_row que puede acceder a N y recibe un row_index.
    // Esta funcion sera usada en los threads, y acceder a N es seguro en este contexto puesto que es solo lectura.
    auto calculate_row = [&](int row_index) {
        for (int j = 0; j < N; j++) {
            int sum = 0;
            for (int k = 0; k < N; k++) {
                sum += matrixA[row_index][k] * matrixB[k][j];
            }
            result[row_index][j] = sum;
        }
    };

    for (int i = 0; i < N; i++) {
        //Construimos el objeto y lo insertamos al final del vector. El hilo inicia inmediatamente despues de creado el objeto
        threads.emplace_back(calculate_row, i);
    }

    //Esperamos a que todos los hilos terminen antes de devolver el resultado
    for (auto& t : threads) {
        t.join();
    }
    return result;
}

void print_matrix(const std::vector<std::vector<int>>& matrix) 
{
    for (const auto& row : matrix) {
        std::cout << "| ";
        for (int x : row) {
            std::cout << x << " |  ";
        }
        std::cout << '\n';
    }
}

int parse_arguments(int argc, char* argv[]) 
{
    if (argc != 2) {
        std::cerr << "Uso: " << argv[0] << " N\n";
        return 1;
    }
    int N = std::stoi(argv[1]);
    if (N < 5 || N > 20) {
        std::cerr << "N debe estar entre 5 y 20\n";
        return 1;
    }
    return N;
}

int main(int argc, char* argv[]) 
{
    int N = parse_arguments(argc, argv);

    constexpr int MIN_VALUE = -32;
    constexpr int MAX_VALUE = 32;

    auto matrixA = generate_random_matrix(N, MIN_VALUE, MAX_VALUE);
    auto matrixB = generate_random_matrix(N, MIN_VALUE, MAX_VALUE);

    auto sequential_result = matrix_multiplication(matrixA, matrixB);
    auto concurrent_result = concurrent_matrix_multiplication(matrixA, matrixB);

    if (sequential_result == concurrent_result) {
        std::cout << "Matrix A:\n";
        print_matrix(matrixA);
        std::cout << "Matrix B:\n";
        print_matrix(matrixB);
        std::cout << "Resultado secuencial:\n";
        print_matrix(sequential_result);
        std::cout << "Resultado concurrente:\n";
        print_matrix(concurrent_result);
    } else {
        std::cout << "Las dos matrices no dieron igual!!\n";
    }
}

Overwriting threads.cpp


In [ ]:
!g++ -std=c++11 -pthread threads.cpp -o program

In [ ]:
!./program 5

Matrix A:
| 27 |  -23 |  -16 |  28 |  1 |  
| -31 |  27 |  28 |  28 |  28 |  
| 20 |  -15 |  -30 |  -4 |  15 |  
| 11 |  -12 |  -18 |  -6 |  -14 |  
| 3 |  21 |  -31 |  -5 |  18 |  
Matrix B:
| 22 |  30 |  23 |  -28 |  29 |  
| 21 |  4 |  -9 |  24 |  2 |  
| -8 |  -15 |  2 |  -24 |  -24 |  
| -31 |  28 |  -31 |  -3 |  13 |  
| -7 |  -16 |  24 |  31 |  14 |  
Resultado secuencial:
| -636 |  1726 |  -48 |  -977 |  1499 |  
| -1403 |  -906 |  -1096 |  1628 |  -761 |  
| 384 |  638 |  1019 |  277 |  1428 |  
| 418 |  608 |  175 |  -580 |  453 |  
| 784 |  211 |  405 |  1737 |  1060 |  
Resultado concurrente:
| -636 |  1726 |  -48 |  -977 |  1499 |  
| -1403 |  -906 |  -1096 |  1628 |  -761 |  
| 384 |  638 |  1019 |  277 |  1428 |  
| 418 |  608 |  175 |  -580 |  453 |  
| 784 |  211 |  405 |  1737 |  1060 |  
